In [ ]:
import json

In [ ]:
# Chargement des fichiers
with open("transcriptions_ordonnées.json", encoding="utf-8") as f:
    transcriptions = json.load(f)

with open("dic_final.json", encoding="utf-8") as f:
    dictionnaire_lemmes = json.load(f)

In [1]:
# Construire forme_vers_lemmes avec priorisation des lemmes ayant plus de variantes
forme_vers_lemmes = {}

for lemme, variantes in dictionnaire_lemmes.items():
    for variante in variantes:
        variante = variante.lower()
        if variante in forme_vers_lemmes:
            lemme_existant = forme_vers_lemmes[variante]
            if len(dictionnaire_lemmes[lemme]) > len(dictionnaire_lemmes[lemme_existant]):
                forme_vers_lemmes[variante] = lemme
        else:
            forme_vers_lemmes[variante] = lemme

def trouver_index_division(mots_auto, lemme_cible, indices_utilisés):
    for i, mot in enumerate(mots_auto):
        if i in indices_utilisés:
            continue
        if forme_vers_lemmes.get(mot.strip(".,!?").lower()) == lemme_cible:
            indices_utilisés.add(i)
            return i
    return None

résultats = []
exemples_problématiques = []
détails_problématiques = []
exemples_traités = []

for exemple in transcriptions:
    if "transcription automatique" not in exemple or "transcription automatique_non_pl" in exemple:
        continue

    mots_auto = exemple["transcription automatique"].split()
    indices_utilisés = set()

    i = 1
    n_segments = 0
    while f"transcription manuelle ({i})" in exemple and f"version correcte en polonais ({i})" in exemple:
        n_segments += 1
        i += 1

    indices_division = []
    indices_utilisés = set()
    ignore_premier = False

    for idx_segment in range(1, n_segments):
        seg_man = exemple[f"transcription manuelle ({idx_segment})"]
        seg_corr = exemple[f"version correcte en polonais ({idx_segment})"]
        dernier_mot = seg_corr.strip().split()[-1].strip(".,!?").lower()

        lemme_cible = forme_vers_lemmes.get(dernier_mot)

        if not lemme_cible:
            dernier_mot_man = seg_man.strip().split()[-1].strip(".,!?").lower()
            lemme_cible = forme_vers_lemmes.get(dernier_mot_man)

        if lemme_cible:
            idx = trouver_index_division(mots_auto, lemme_cible, indices_utilisés)
        else:
            idx = None

        lemmes_suivants = []
        if idx is None and idx_segment + 1 <= n_segments:
            prochain_seg_corr = exemple.get(f"version correcte en polonais ({idx_segment + 1})", "")
            prochain_seg_man = exemple.get(f"transcription manuelle ({idx_segment + 1})", "")
            mots_suivants = (prochain_seg_corr or prochain_seg_man).strip().split()[:2]

            for mot in mots_suivants:
                mot_clean = mot.strip(".,!?").lower()
                lemme = forme_vers_lemmes.get(mot_clean)
                if lemme:
                    lemmes_suivants.append(lemme)

            for i in range(len(mots_auto) - 1):
                if i in indices_utilisés or i + 1 in indices_utilisés:
                    continue
                l1 = forme_vers_lemmes.get(mots_auto[i].strip(".,!?").lower())
                l2 = forme_vers_lemmes.get(mots_auto[i + 1].strip(".,!?").lower())
                if len(lemmes_suivants) == 2 and l1 == lemmes_suivants[0] and l2 == lemmes_suivants[1]:
                    idx = i
                    indices_utilisés.add(i)
                    indices_utilisés.add(i + 1)
                    break

        if idx is None:
            if idx_segment == 1:
                indices_division.append(0)
                ignore_premier = True
                continue
            else:
                exemples_problématiques.append(exemple["numéro d'apprenant"])
                détails_problématiques.append({
                    "numéro d'apprenant": exemple["numéro d'apprenant"],
                    "problème": "Aucun lemme ou combinaison trouvée",
                    "dernier_mot_corr": dernier_mot,
                    "transcription automatique": exemple["transcription automatique"]
                })
                résultats.append(exemple)
                break

        if len(lemmes_suivants) == 2 and idx in range(len(mots_auto) - 1):
            indices_division.append(idx)
        else:
            indices_division.append(idx + 1)

    else:
        précédent = 0
        segments_auto = []
        for idx in indices_division:
            segments_auto.append(" ".join(mots_auto[précédent:idx]))
            précédent = idx
        segments_auto.append(" ".join(mots_auto[précédent:]))

        nouvel_exemple = {
            "numéro d'apprenant": exemple["numéro d'apprenant"],
            "pays": exemple["pays"]
        }

        for j in range(1, n_segments + 1):
            if ignore_premier and j == 1:
                nouvel_exemple[f"transcription automatique ({j})"] = ""
            else:
                nouvel_exemple[f"transcription automatique ({j})"] = segments_auto[j - 1]
            nouvel_exemple[f"transcription manuelle ({j})"] = exemple[f"transcription manuelle ({j})"]
            nouvel_exemple[f"version correcte en polonais ({j})"] = exemple[f"version correcte en polonais ({j})"]

        for clé, valeur in exemple.items():
            if clé not in nouvel_exemple and clé != "transcription automatique":
                nouvel_exemple[clé] = valeur

        résultats.append(nouvel_exemple)
        exemples_traités.append(exemple["numéro d'apprenant"])

# Affichage
nombre_total = len(transcriptions)
nombre_traités = len(exemples_traités)
nombre_problématiques = len(exemples_problématiques)

print(f"Exemples traités : {nombre_traités} ({nombre_traités / nombre_total * 100:.2f}%)")
print(f"Exemples problématiques : {nombre_problématiques} ({nombre_problématiques / nombre_total * 100:.2f}%)")

print("\nDétails des problèmes rencontrés :")
for détail in détails_problématiques:
    print(json.dumps(détail, ensure_ascii=False, indent=2))

Exemples traités : 62 (69.66%)
Exemples problématiques : 12 (13.48%)

Détails des problèmes rencontrés :
{
  "numéro d'apprenant": "1117",
  "problème": "Aucun lemme ou combinaison trouvée",
  "dernier_mot_corr": "prawo",
  "transcription automatique": "jestem blisko ulicy niska i mieszkam w szpitalu więc nie jestem w porządku przeszłam proces i proces proces proces i obok sklepu na lewo na ulicę dobrą i na lewo jest nr 4"
}
{
  "numéro d'apprenant": "5117",
  "problème": "Aucun lemme ou combinaison trouvée",
  "dernier_mot_corr": "sklepu",
  "transcription automatique": "proszę iść prościej do szpitalu do szpitalu skręcić się w prawo na ulicę słowackiego a potem proszę iść prościej do ulicy dobrej 4 a potem proszę iść prościej do ulicy słowackiego a potem proszę iść prościej do ulicy słowackiego a potem proszę iść prościej do ulicy słowackiego"
}
{
  "numéro d'apprenant": "5106",
  "problème": "Aucun lemme ou combinaison trouvée",
  "dernier_mot_corr": "szpitala",
  "transcription aut

In [2]:
# Résultats bien segmentés
résultats_bien = [ex for ex in résultats if ex["numéro d'apprenant"] in exemples_traités]

# Ajouter les exemples problématiques (non segmentés)
transcriptions_dict = {ex["numéro d'apprenant"]: ex for ex in transcriptions}
for numéro in exemples_problématiques:
    if numéro in transcriptions_dict:
        résultats_bien.append(transcriptions_dict[numéro])

# Ajouter les exemples avec "transcription automatique_non_pl"
for exemple in transcriptions:
    if "transcription_automatique_non_pl" in exemple:
        nouveau_exemple = {
            "numéro d'apprenant": exemple["numéro d'apprenant"],
            "pays": exemple.get("pays", ""),
            "transcription_automatique_non_pl": exemple["transcription_automatique_non_pl"]
        }

        # Copier tous les segments manuels et corrects
        i = 1
        while f"transcription manuelle ({i})" in exemple and f"version correcte en polonais ({i})" in exemple:
            nouveau_exemple[f"transcription manuelle ({i})"] = exemple[f"transcription manuelle ({i})"]
            nouveau_exemple[f"version correcte en polonais ({i})"] = exemple[f"version correcte en polonais ({i})"]
            i += 1

        # Ajouter toutes les autres clés non présentes déjà (métadonnées, erreurs, etc.)
        for clé, valeur in exemple.items():
            if clé not in nouveau_exemple:
                nouveau_exemple[clé] = valeur

        résultats_bien.append(nouveau_exemple)

# Sauvegarder le fichier final
with open("transcriptions_segmentées_final.json", "w", encoding="utf-8") as f:
    json.dump(résultats_bien, f, ensure_ascii=False, indent=2)

print(f"Fichier final enregistré avec {len(résultats_bien)} exemples.")

Fichier final enregistré avec 89 exemples.
